In [1]:
import numpy as np
import matplotlib.pyplot as plt
import math
from sklearn.linear_model import HuberRegressor
import numpy.random as rgt
from robust_mean import huberReg


In [ ]:
# Assuming huberReg is your custom class for noisy Huber regression

def l2_distance(vec1, vec2):
    return np.sqrt(np.sum((vec1 - vec2) ** 2))

def run_experiment(n, d, repetitions, eta, epsilon, T, delta, epsilon2=None):
    errors_noisy = []
    errors_ordinary = []
    if epsilon2 is not None:
        errors_noisy2 = []
    
    for _ in range(repetitions):
        # Generate sample data
        X = np.random.uniform(-1/np.sqrt(d), 1/np.sqrt(d), (n, d))
        truebeta = np.random.normal(0, 1, d)
        truebeta /= np.linalg.norm(truebeta)
        #Y = X.dot(truebeta) + np.random.normal(0, 4, n)+1
        Y = X.dot(truebeta) + rgt.standard_t(2, n) +1
        beta0 = np.zeros(d+1)

        # Noisy Huber Regression
        noisy_huber_regression = huberReg(X, Y)
        noisy_huber_result = noisy_huber_regression.noisy_huber_reg_lowdim(beta0=beta0, epsilon=epsilon, T=T, delta=delta, eta=eta)
        if epsilon2 is not None:
            noisy_huber_result2 = noisy_huber_regression.noisy_huber_reg_lowdim(beta0=beta0, epsilon=epsilon2, T=T, delta=delta, eta=eta)
            error_noisy2 = l2_distance(noisy_huber_result2[0][1:], truebeta)
            errors_noisy2.append(error_noisy2)
        error_noisy = l2_distance(noisy_huber_result[0][1:], truebeta)  # Assuming the first return value is the regression result
        errors_noisy.append(error_noisy)

        # Ordinary Huber Regression
        huber_reg = HuberRegressor().fit(X, Y)
        ordinary_huber_coefficients = huber_reg.coef_
        error_ordinary = l2_distance(ordinary_huber_coefficients, truebeta)
        errors_ordinary.append(error_ordinary)
        #error_between = l2_distance(ordinary_huber_coefficients,noisy_huber_result[0][1:])
        #errors_between.append(error_between)
    if epsilon2 is not None:
        return np.mean(errors_noisy),np.mean(errors_noisy2),np.mean(errors_ordinary)
    else:
        return np.mean(errors_noisy), np.mean(errors_ordinary)

In [ ]:
# Parameters
d = 5
eta = 1
delta = 0.001
#T=10
epsilon = 0.5
epsilon2 = 0.3
repetitions = 100
sample_sizes = np.array(range(2000, 30000, 500))  # Adjust sample sizes as needed

# Running the experiment and plotting
mean_errors_noisy = []
mean_errors_ordinary = []
mean_errors_noisy2 = []
for n in sample_sizes:
    errors_noisy = []
    errors_noisy2 = []
    errors_ord = []
    T=int(np.log(n))
    for m in range(repetitions):
        X = np.random.uniform(-1/np.sqrt(d), 1/np.sqrt(d), (n, d))
        truebeta = np.random.normal(0, 1, d)
        truebeta /= np.linalg.norm(truebeta)
        #Y = X.dot(truebeta) + np.random.normal(0, 4, n)+1
        Y = X.dot(truebeta) + rgt.standard_t(2.2, n) +1
        beta0 = np.zeros(d+1)

        # Noisy Huber Regression1
        noisy_huber_regression = huberReg(X, Y)
        noisy_huber_result = noisy_huber_regression.noisy_huber_reg_lowdim(beta0=beta0, epsilon=epsilon, T=T, delta=delta, eta=eta)
        errors_noisy.append(l2_distance(noisy_huber_result[0][1:], truebeta))
        # Noisy Huber Regression2
        noisy_huber_regression = huberReg(X, Y)
        noisy_huber_result2 = noisy_huber_regression.noisy_huber_reg_lowdim(beta0=beta0, epsilon=epsilon2, T=T, delta=delta, eta=eta)
        errors_noisy2.append(l2_distance(noisy_huber_result2[0][1:], truebeta))
        #Ordinary Huber
        huber_reg = HuberRegressor().fit(X, Y)
        ordinary_huber_coefficients = huber_reg.coef_
        errors_ord.append(l2_distance(ordinary_huber_coefficients, truebeta))
    mean_errors_noisy.append(np.mean(errors_noisy))
    mean_errors_noisy2.append(np.mean(errors_noisy2))
    mean_errors_ordinary.append(np.mean(errors_ord))


        

In [ ]:
plt.plot(sample_sizes, np.log(mean_errors_noisy), label='Noisy Huber Regression_epsilon=0.5')
plt.plot(sample_sizes, np.log(mean_errors_noisy2), label='Noisy Huber Regression_epsilon=0.3')
plt.plot(sample_sizes, np.log(mean_errors_ordinary), label='Ordinary Huber Regression')
#plt.plot(sample_sizes, mean_errors_between, label='error between estimator')
plt.xlabel('Sample Size')
plt.ylabel('Log Estimation Error')
plt.title('Comparison of Regression Methods')
plt.legend()
#plt.savefig('my_plot_t22.png', dpi=300)
plt.show()

In [2]:
n, d = 1000, 1000
X = np.random.randn(n, d)
true_beta = np.zeros(d)
true_beta[:5] = np.random.randn(5)  # Sparse true coefficients
true_beta = true_beta/np.linalg.norm(true_beta)
Y = X.dot(true_beta) + np.random.randn(n)

In [3]:
reg = huberReg(X,Y)

In [4]:
eta = 1
delta = 0.001
T=int(np.log(1000))
epsilon = 0.5

In [6]:
beta0=np.zeros(d+1)

In [7]:
beta_final, diagnostics, beta_sequence = reg.noisy_huber_reg_highdim(beta0=beta0,epsilon=epsilon, T=T, delta=delta, eta=eta, s=5 )

In [8]:
beta_final

array([0., 0., 0., 0., 0., 0., 0.])

In [25]:
beta_sequence[:,6][0:6]

array([-0.11245236,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ])